In [411]:
import numpy as np

In [412]:
obs = ('normal', 'cold', 'dizzy')
states = ('Healthy', 'Fever')
start_p = {'Healthy': 0.6,
           'Fever': 0.4}
trans_p = {'Healthy': {'Healthy': 0.7, 'Fever': 0.3},
           'Fever': {'Healthy': 0.4, 'Fever': 0.6}}
emit_p = {'Healthy': {'normal': 0.5, 'cold': 0.4, 'dizzy': 0.1},
          'Fever': {'normal': 0.1, 'cold': 0.3, 'dizzy': 0.6}}

In [413]:
def viterbi(obs, 
            states, 
            start_p,
            observations,
            trans_p,
            emit_p):
    V = [{}]
    # We are going to iterate through all the observations,
    # starting from normal. 
    # Calculate the probability for the given states.
    for i, st in enumerate(states):
        V[0][st] = {'prob': start_p[st] * emit_p[st][obs[0]], 
                    'prev': None} 
        
    # For the rest of the observations.
    for t in range(1, len(obs)):
        V.append({})
        for st in states:
            # Take the first state and calculate the probability.
            # Use it as the initial guess.
            max_tr_prob = V[t-1][states[0]]['prob'] * trans_p[states[0]][st]
            prev_st_selected = states[0]

            # Now loop through the remaining states and compare the probability.
            # Take the one with the highest probability.
            for prev_st in states[1:]:
                tr_prob = V[t-1][prev_st]['prob'] * trans_p[prev_st][st]
                if tr_prob > max_tr_prob:
                    max_tr_prob = tr_prob
                    prev_st_selected = prev_st

            max_prob = max_tr_prob * emit_p[st][obs[t]]
            V[t][st] = {'prob': max_prob,
                        'prev': prev_st_selected}
    for line in dptable(V):
        print(line)

    opt = []
    # The highest probability.
    max_prob = max(value['prob']
                   for value in V[-1].values())
    max_prob
    # Get the most probable state and its backtrack.
    for st, data in V[-1].items():
        if data['prob'] == max_prob:
            opt.append(st)
            previous = st
            break
    # Follow the backtrack until the first observation. 
    # Minus 2, since we exclude the last result.
    for t in range(len(V) - 2, -1, -1):
        opt.insert(0, V[t+1][previous]['prev'])
        previous = V[t+1][previous]['prev']
    return f'The step of states are {" ".join(opt)} with the probability of {max_prob}'

In [414]:
def dptable(V):
    header = [f'{"":12s}'] + [f'{str(i):12s}' for i in range(len(V))]
    yield ' '.join(header)
    
    for state in V[0].keys():
        row = [f'{state:12s}'] + [f'{str(round(d[state]["prob"], 5)):12s}' 
                                  for d in V]
        yield ' '.join(row)

In [415]:
viterbi(obs, states, start_p, obs, trans_p, emit_p)

             0            1            2           
Healthy      0.3          0.084        0.00588     
Fever        0.04         0.027        0.01512     


'The step of states are Healthy Healthy Fever with the probability of 0.01512'

In [444]:
viterbi([obs[i] for i in [1,2,0,1,2]]), states, start_p, obs, trans_p, emit_p)

SyntaxError: invalid syntax (<ipython-input-444-11503c346e97>, line 1)

In [417]:
import numpy as np

# http://www.blackarbs.com/blog/introduction-hidden-markov-models-python-networkx-sklearn/2/9/2017

In [430]:
def viterbi(O, S, pi, A, B):
    """
    O: observation space
    S: state space
    pi: initial probabilities
    y: sequence of observations
    A: transition matrix. 2d array with A[from_state][to_state]
    B: emission matrix. 2d array with B[state_i][observation_j]
    """
    
    K, T = len(S), len(O)
    
    # Stores the probability of the most likely path so far.
    T1 = np.zeros((K, T))
    
    # Stores the argmax of the most probable path.
    T2 = np.zeros((K, T))
    
    # Initial states for initial observation.
    T1[:, 0] = pi * B[:, O[0]]
    
    for j in range(1, T):
        for i in range(K):
            p = T1[:, j-1] * A[:, i]
            T1[i, j] = np.max(p * B[i, O[j]])
            T2[i, j] = np.argmax(p * A[:, i])
    
    # Paths.
    z = np.zeros(T).astype(np.int)
    z[-1] = np.argmax(T1[:, -1])
    
#     prev = z[-1]
    for j in range(T-2, -1, -1):
        z[j] = T2[z[j+1], j]
#         z[j] = T2[z[j+1], prev]
#         prev = z[j]
    return z, T1, T2

In [431]:
states_dict = {
    0: 'healthy', 1: 'sick'
}
obs = [1,1,2,1,0,1,2,1,0,2,2,0,1,0,1] # 0: sleeping, 1: eating, 2: pooping
obs_dict = {
    0: 'sleeping', 
    1: 'eating',
    2: 'pooping'
}
states = (0, 1) # healthy, sick
start_p = np.array([0.5, 0.5])
trans_p = np.array([[0.7, 0.3],
                    [0.4, 0.6]])
emit_p = np.array([[0.2, 0.6, 0.2],
                   [0.4, 0.1, 0.5]])

In [432]:
paths, _, _ = viterbi(obs, states, start_p, trans_p, emit_p)
print(paths)
for i, p in enumerate(paths):
    print(f'{obs_dict[obs[i]]:10} -> {states_dict[p]}')

[0 0 0 0 0 0 0 0 0 1 1 1 0 0 0]
eating     -> healthy
eating     -> healthy
pooping    -> healthy
eating     -> healthy
sleeping   -> healthy
eating     -> healthy
pooping    -> healthy
eating     -> healthy
sleeping   -> healthy
pooping    -> sick
pooping    -> sick
sleeping   -> sick
eating     -> healthy
sleeping   -> healthy
eating     -> healthy


In [433]:
obs = (0, 1, 2) # 0: normal, 1: cold, 2: dizzy
states = (0, 1) # 0: Healthy, 1: Fever

# start_p[initial_state]
start_p = np.array([0.6, 0.4])

# trans_p[from_state][to_state]
trans_p = np.array([[0.7, 0.3],
                    [0.4, 0.6]])

# emit_p[state][observation]
emit_p = np.array([[0.5, 0.4, 0.1],
                   [0.1, 0.3, 0.6]])

In [437]:
paths, result, _ = viterbi(obs, states, start_p, trans_p, emit_p)
print(paths)
print(result)

[0 0 1]
[[0.3     0.084   0.00588]
 [0.04    0.027   0.01512]]


In [442]:
paths, result, _ = viterbi((1,2,0,1,2), states, start_p, trans_p, emit_p)
print(paths)
print(result)

[0 1 1 1 1]
[[2.40000e-01 1.68000e-02 8.64000e-03 2.41920e-03 1.69344e-04]
 [1.20000e-01 4.32000e-02 2.59200e-03 7.77600e-04 4.35456e-04]]


In [443]:
np.argmax([1,2,35])

2

In [425]:
A = np.array([[1,2,3], [4,5,6]])
np.argmax(A, axis=0) # Find the max index for the given columns.

array([1, 1, 1])

In [426]:
np.argmax(A, axis=1) # Find the max index for the given rows.

array([2, 2])

In [427]:
np.max(A, axis=0) # Find the max index for the given rows.

array([4, 5, 6])

In [428]:
A = np.array([1,2,3])
B = np.array([4,5,6])
np.multiply(A, B) * 4

array([16, 40, 72])